In [1]:
# Imports
import pandas as pd
from anomaly_tools import HDBAnomalyDetector
from database_tools import PandasDBReader
from fh_config import regression_vars, response_var

In [50]:
# We need to create a PandasDBReader to interface with anomaly detectors.
states = ['FL']
specialty = ['Cardiology']
pdb_reader = PandasDBReader("./config.yaml", states, specialty)

In [51]:
# Next, we compute the outlier counts from HDBSCAN.
hdb = HDBAnomalyDetector(regression_vars, response_var, pdb_reader.d_f, use_response_var=True)
min_cluster_size = 10  # Going with 15 for now, it's not too hard to update this later.
hdb.get_outlier_scores(min_size=min_cluster_size)

In [52]:
# We need a DataFrame that contains the actual outlier counts.
counted_df = hdb.get_most_frequent()

In [58]:
print(counted_df.head(20)['last_name'].values)

['QAMAR' 'SEDILLO' 'PAL' 'LEW' 'REDDY' 'CHALASANI' 'LEE' 'KHATIB'
 'LIEBERMAN' 'RUGGIERI' 'RANDALL' 'VANROY' 'ALI' 'VAN DEN BERG' 'MANUBENS'
 'BREDLAU' 'STEIN' 'ATTANTI' 'FERNS' 'GUMMADI']


In [62]:
top_20_dict = {}
for size in [10, 15, 20, 40, 80, 100, 150]:
    hdb.get_outlier_scores(min_size=size)
    counted_df = hdb.get_most_frequent()
    top_20_dict[size] = list(counted_df.head(20)['last_name'].values)
print(top_20_dict)

{80: ['QAMAR', 'PAL', 'SEDILLO', 'GOEL', 'CHALASANI', 'LEW', 'REDDY', 'ATTANTI', 'MANUBENS', 'BAJAJ', 'GUMMADI', 'VAN DEN BERG', 'PRESSMAN', 'RANDALL', 'LIEBERMAN', 'UPADYA', 'STEIN', 'SESHADRI', 'KHATIB', 'GARCIA'], 20: ['QAMAR', 'SEDILLO', 'PAL', 'LEW', 'CHALASANI', 'ATTANTI', 'KHATIB', 'REDDY', 'GOEL', 'BREDLAU', 'GUALA', 'LEE', 'VAN DEN BERG', 'GARCIA', 'RANDALL', 'MANUBENS', 'BHATTA', 'RUGGIERI', 'ZACHARIAH', 'ROSENFIELD'], 150: ['QAMAR', 'SEDILLO', 'LEW', 'PAL', 'CHALASANI', 'KHATIB', 'ATTANTI', 'REDDY', 'UPADYA', 'GOEL', 'GUMMADI', 'LEE', 'TAUSSIG', 'WILLARD', 'GUALA', 'SINGH', 'ANDERSON', 'MANUBENS', 'DUBLIN', 'YELAMANCHI'], 40: ['QAMAR', 'PAL', 'SEDILLO', 'CHALASANI', 'ATTANTI', 'LEW', 'UPADYA', 'MANUBENS', 'REDDY', 'VAN DEN BERG', 'KHATIB', 'RUGGIERI', 'GOEL', 'ZACHARIAH', 'RANDALL', 'BHATTA', 'ALONSO', 'SECKLER', 'BREDLAU', 'LIEBERMAN'], 100: ['QAMAR', 'SEDILLO', 'LEW', 'CHALASANI', 'PAL', 'KHATIB', 'REDDY', 'ATTANTI', 'UPADYA', 'GOEL', 'GUMMADI', 'TAUSSIG', 'LEE', 'GUALA', 

In [7]:
# From this DataFrame we create a sliced DataFrame containing the columns for the new table.
npi_column = list(counted_df.index)
state_column = [geo_info['state'] for geo_info in counted_df['address'].values]
lastname_column = counted_df['last_name']
outlier_count_column = counted_df['outlier_count']
provider_column = [specialty[0] for i in range(len(npi_column))]
for_new_table_data = {'state': state_column, 'lastname': lastname_column, 'provider_type': provider_column, 'outlier_count': outlier_count_column}
for_new_table = pd.DataFrame(data=for_new_table_data, index=npi_column)
for_new_table.to_csv("/home/dan/tx_im.csv")

In [23]:
# Next step: Testing that everything worked
import psycopg2
import yaml

In [25]:
with open("./config.yaml", 'r') as f:
    db_config = yaml.load(f)
con = psycopg2.connect(database=db_config['database_name'],
                       user=db_config['user_name'],
                       password=db_config['password'])
new_query = "SELECT * from provider_anomaly_counts_hdb"
test_df = pd.read_sql_query(new_query, con)
print(test_df.head())

          npi state   lastname provider_type  outlier_count
0  1033145487    FL      QAMAR    Cardiology             48
1  1982680179    FL    SEDILLO    Cardiology             34
2  1376588582    FL        PAL    Cardiology             29
3  1861485757    FL        LEW    Cardiology             21
4  1760477269    FL  CHALASANI    Cardiology             21
